# Загрузка и первичный осмотр данных
 В этом разделе:
 1. Импортируем библиотеку pandas для работы с таблицами и numpy для численных операций.
 2. Загружаем CSV-файл "emails.csv" в DataFrame — табличную структуру, удобную для анализа.
 3. Выводим первые 10 строк, чтобы проверить, что данные корректно загружены и понять формат.
    Ожидаем, что в таблице есть столбцы:
      - 'text'  : текст письма
      - 'spam'  : метка 1 (spam) или 0 (ham)


In [2]:
import numpy as np
import pandas as pd

emails = pd.read_csv('emails.csv')
print("Первые 10 писем для проверки:")
print(emails.head(10))



Первые 10 писем для проверки:
                                                text  spam
0  Subject: naturally irresistible your corporate...     1
1  Subject: the stock trading gunslinger  fanny i...     1
2  Subject: unbelievable new homes made easy  im ...     1
3  Subject: 4 color printing special  request add...     1
4  Subject: do not have money , get software cds ...     1
5  Subject: great nnews  hello , welcome to medzo...     1
6  Subject: here ' s a hot play in motion  homela...     1
7  Subject: save your money buy getting this thin...     1
8  Subject: undeliverable : home based business f...     1
9  Subject: save your money buy getting this thin...     1


# Преобразование текста письма в набор уникальных слов

 Задача: перейти от неструктурированного текста к списку признаков.
 Для каждого письма:
 1. Переводим текст в нижний регистр — чтобы слова "Hello" и "hello"
    считались одинаковыми.
 2. Разбиваем строку по пробелам на слова.
 3. Используем set() для получения уникальных слов — нам важно
    только наличие слова, а не его количество.
 4. Сохраняем этот список уникальных слов в новом столбце 'words'.


In [3]:
def process_email(text):
    text = text.lower()                     # делаем буквы строчными
    words = text.split()                    # разбиваем по пробелам
    unique = list(set(words))               # оставляем только уникальные
    return unique

# Применяем функцию к каждому отзыву и создаем новый столбец
emails['words'] = emails['text'].apply(process_email)
print("После обработки текста (столбец 'words'):")
print(emails.head(10))

После обработки текста (столбец 'words'):
                                                text  spam  \
0  Subject: naturally irresistible your corporate...     1   
1  Subject: the stock trading gunslinger  fanny i...     1   
2  Subject: unbelievable new homes made easy  im ...     1   
3  Subject: 4 color printing special  request add...     1   
4  Subject: do not have money , get software cds ...     1   
5  Subject: great nnews  hello , welcome to medzo...     1   
6  Subject: here ' s a hot play in motion  homela...     1   
7  Subject: save your money buy getting this thin...     1   
8  Subject: undeliverable : home based business f...     1   
9  Subject: save your money buy getting this thin...     1   

                                               words  
0  [creativeness, subject:, ,, full, good, logo, ...  
1  [continuant, esmark, hall, mcdougall, subject:...  
2  [new, website, home, fixed, subject:, rate, ea...  
3  [printing, ), form, subject:, version, ,, 9170...  


# Оценка базовой вероятности (априор) спама
 Здесь считаем, какая доля писем в целом является спамом.
 Это P(spam) — априорная вероятность для модели.


In [4]:
num_emails = len(emails)
num_spam = emails['spam'].sum()
num_ham = num_emails - num_spam
print(f"Всего писем: {num_emails}")
print(f"Спам-писем: {num_spam}")
print(f"Вероятность спама P(spam): {num_spam/num_emails:.4f}\n")

Всего писем: 5728
Спам-писем: 1368
Вероятность спама P(spam): 0.2388



# Обучение простейшей наивной байесовской модели по словам

 Модель хранит для каждого слова два счетчика:
   model[word]['spam'] = количество спам-писем, где встречается слово
   model[word]['ham']  = количество не-спам писем, где слово встречается
 Для корректности используем аддитивную (Лапласову) сглаженность: начинаем
 счетчики с 1, чтобы ни одна вероятность не стала нулевой.


In [5]:
model = {}  # словарь вида {word: {'spam': count, 'ham': count}}
for _, row in emails.iterrows():
    is_spam = row['spam'] == 1
    for word in row['words']:
        if word not in model:
            model[word] = {'spam': 1, 'ham': 1}  # инициализация сглаживанием
        if is_spam:
            model[word]['spam'] += 1
        else:
            model[word]['ham'] += 1

# Посмотрим на пару примеров слов
print("Пара примеров из модели:")
for w in ['lottery', 'sale']:
    print(f"{w}: {model.get(w)}")

Пара примеров из модели:
lottery: {'spam': 9, 'ham': 1}
sale: {'spam': 39, 'ham': 42}


# Прогноз для одного слова (P(spam|word))

 По формуле Байеса для одного слова (игнорируя нормировочный множитель):
   P(spam|word) ∝ P(word|spam)*P(spam)
 Но мы возвращаем простое отношение частот:
   count_spam / (count_spam + count_ham).


In [6]:
def predict_bayes(word):
    word = word.lower()
    counts = model.get(word)
    if counts is None:
        return None  # слово не встречалось в обучении
    return counts['spam'] / (counts['spam'] + counts['ham'])

print("P(spam|'lottery') =", predict_bayes('lottery'))
print("P(spam|'sale')    =", predict_bayes('sale'))

P(spam|'lottery') = 0.9
P(spam|'sale')    = 0.48148148148148145


# Полный наивный байес для всего письма

 Предполагаем независимость слов. Для письма W = {w1,w2,...}:
   P(spam|W) ∝ P(spam)*∏ P(wi|spam)
   P(ham|W)  ∝ P(ham)*∏ P(wi|ham)
 Сравниваем эти два произведения (нормировка не нужна для решения).


In [7]:
def predict_naive_bayes(email_text):
    # подготовка
    email_text = email_text.lower()
    words = set(email_text.split())

    # априорные
    p_spam = num_spam / num_emails
    p_ham = num_ham / num_emails

    # умножаем условные вероятности
    for word in words:
        if word in model:
            counts = model[word]
            # P(word|spam) ≈ counts['spam']/num_spam
            p_spam *= counts['spam'] / num_spam
            # P(word|ham)
            p_ham *= counts['ham'] / num_ham

    # нормируем
    total = p_spam + p_ham
    return p_spam/total if total > 0 else 0.5

# Примеры работы
tests = [
    'lottery sale',
    'Hi mom how are you',
    'enter the lottery to win three million dollars',
    'meet me at the lobby of the hotel at nine am'
]
for t in tests:
    print(f"P(spam|'{t}') = {predict_naive_bayes(t):.4f}")

P(spam|'lottery sale') = 0.9638
P(spam|'Hi mom how are you') = 0.1255
P(spam|'enter the lottery to win three million dollars') = 0.9995
P(spam|'meet me at the lobby of the hotel at nine am') = 0.0001
